# 코틀린 반공변성(Contravariance)
- 상위 타입을 하위 타입 자리에 허용하는 것
- `Rose <: Flower` 라면, `FlowerCare<Flower>`를 `FlowerCare<Rose>` 자리에 넣을 수 있게 만드는 것.


In [5]:
open class Flower(val name: String)
class Rose : Flower("Rose")
class Daffodil : Flower("Daffodil")

interface FlowerCare<T> {
    fun prune(flower: T)
}

class Garden<T : Flower>(
    private val flowers: List<T>,
    private val flowerCare: FlowerCare<T>
) {
    fun pickFlower(index: Int): T = flowers[index]
    fun tendFlower(index: Int) = flowerCare.prune(flowers[index])
}

// Rose 전용 관리자
val roseTender = object : FlowerCare<Rose> {
    override fun prune(flower: Rose) {
        println("I'm pruning a ${flower.name}")
    }
}

// Daffodil 전용 관리자 - 거의 동일한 로직!
val daffodilTender = object : FlowerCare<Daffodil> {
    override fun prune(flower: Daffodil) {
        println("I'm pruning a ${flower.name}")
    }
}

val roseGarden = Garden(listOf(Rose(), Rose()), roseTender)
val daffodilGarden = Garden(listOf(Daffodil(), Daffodil()), daffodilTender)

In [8]:
interface FlowerCare<in T> {  // in 키워드 추가
    fun prune(flower: T)
}

// 이제 하나의 구현체로 모든 정원에서 사용 가능!
val flowerTender = object : FlowerCare<Flower> {
    override fun prune(flower: Flower) {
        println("I'm tending a ${flower.name}")
    }
}

val roseGarden1 = Garden(listOf(Rose(), Rose()), flowerTender)     // ✅ 작동!
val daffodilGarden1 = Garden(listOf(Daffodil()), flowerTender)    // ✅ 작동!

org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[8], line 12, column 50: Type mismatch: inferred type is Line_11_jupyter.FlowerCare<Line_8_jupyter.Flower> but Line_8_jupyter.FlowerCare<TypeVariable(T)> was expected
at Cell In[8], line 13, column 50: Type mismatch: inferred type is Line_11_jupyter.FlowerCare<Line_8_jupyter.Flower> but Line_8_jupyter.FlowerCare<TypeVariable(T)> was expected

## 3. In Position vs Out Position
- Out Position: 함수의 반환 타입 - 데이터를 "내보내는" 위치 : 위험 ❌
- In Position: 함수의 매개변수 타입 - 데이터를 "받아들이는" 위치 : 안전 ✅
#### 제약 이유
- 타입 안정성 보장

In [10]:
class Garden<T : Flower>(
    private val flowers: List<T>,
    private val flowerCare: FlowerCare<T>
) {
    fun pickFlower(index: Int): T = flowers[index]
    fun tendFlower(index: Int) = flowerCare.prune(flowers[index])
}

// 만약 반공변 인터페이스에서 반환이 가능하다면...
interface FlowerCare<in T> {
    fun prune(flower: T)
    fun pick(): T  // 가정: 이것이 허용된다면...
}

val flowerTender = object : FlowerCare<Flower> {
    override fun prune(flower: Flower) { /* ... */ }
    override fun pick(): Flower = Flower("Generic") // 일반적인 꽃 반환
}

val roseGarden: Garden<Rose> = Garden(listOf(Rose), flowerTender)
val rose: Rose = roseGarden.pickFlower(0) // Rose를 기대하지만 Flower를 받음!

org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[10], line 12, column 17: Type parameter T is declared as 'in' but occurs in 'out' position in type T

### 실제 예시
```kotlin
interface EventHandler<in T> {
    fun handle(event: T)
}

class UIEventHandler : EventHandler<UIEvent> {
    override fun handle(event: UIEvent) {
        println("Handling UI event: $event")
    }
}

// 더 구체적인 이벤트 타입에서도 사용 가능
val buttonHandler: EventHandler<ButtonClickEvent> = UIEventHandler() // ✅
```

### 함수 타입에서 사용
```kotiln
// 함수 타입은 매개변수에 대해 반공변
val flowerProcessor: (Flower) -> Unit = { flower ->
    println("Processing ${flower.name}")
}

val roseProcessor: (Rose) -> Unit = flowerProcessor // ✅ 가능!
// Flower를 처리할 수 있으면 Rose도 처리 가능 (Rose는 Flower이므로)
```

## 공변성 선언 시점
#### 선언 시점
```kotlin
// 인터페이스/클래스 선언할 때 variance 지정
interface Producer<out T> { fun produce(): T }
interface Consumer<in T> { fun consume(item: T) }
```
#### 사용 시점
```kotlin
// 사용할 때 variance 지정 (Java 스타일)
fun processFlowers(care: FlowerCare<in Rose>) {
    // Rose와 그 상위 타입을 받는 FlowerCare만 허용
}
```

## 언제 사용?
- Consumer 역할 (데이터를 받아서 처리)
- 함수의 매개변수 타입
- 이벤트 핸들러, 콜백 함수
- 상위 타입 하나로 여러 하위 타입을 처리하고 싶을 때

```kotlin
// 정렬 비교자
interface Comparator<in T> {
    fun compare(a: T, b: T): Int
}

val flowerComparator: Comparator<Flower> = Comparator { a, b ->
    a.name.compareTo(b.name)
}

// Rose 리스트도 정렬 가능
val roses: List<Rose> = listOf(/*...*/)
roses.sortedWith(flowerComparator) // ✅ 작동!
```
## 사용하지 말아야할 경우
- Producer 역할 (데이터를 생성/반환)
- 반환 타입이 중요한 경우
- 타입 안전성을 엄격히 지켜야 하는 경우

### 공변, 반 공변 비교
| 구분    | 공변성 (Covariance) | 반공변성 (Contravariance) |
| ----- | ---------------- | --------------------- |
| 키워드   | `out`            | `in`                  |
| 역할    | Producer (생산자)   | Consumer (소비자)        |
| 허용 위치 | 반환 타입 (Out)      | 매개변수 (In)             |
| 예시    | `List<out T>`    | `Comparator<in T>`    |
